<a href="https://colab.research.google.com/github/moustafa-7/ECG-web-app/blob/master/ECG_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install wfdb
!pip install pywavelets

     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 163kB 26.7MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100368 sha256=c34298bc460089e36030304af187c0c2cd6b238d0098dc75fd8434953a4acdd0
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb


In [0]:
import numpy as np
import sys, os
import wfdb
import pywt
import pickle as pk
from collections import Counter

In [0]:
data_names = ['100', '101', '102', '103', '104', '105', '106', '107', 
              '108', '109', '111', '112', '113', '114', '115', '116', 
              '117', '118', '119', '121', '122', '123', '124', '200', 
              '201', '202', '203', '205', '207', '208', '209', '210', 
              '212', '213', '214', '215', '217', '219', '220', '221', 
              '222', '223', '228', '230', '231', '232', '233', '234']

In [0]:
wid = 100

labels =  
X = []
Y = []
for d in data_names:
  r=wfdb.rdrecord('drive/My Drive/ECG_project/files/mitdb/1.0.0/'+d)
  ann=wfdb.rdann('drive/My Drive/ECG_project/files/mitdb/1.0.0/'+d, 'atr', return_label_elements=['label_store', 'symbol'])
  sig = np.array(r.p_signal[:,0])
  sig_len = len(sig)
  sym = ann.symbol
  pos = ann.sample
  beat_len = len(sym)
  for i in range(1,beat_len-1):
    if sym[i] in labels: 
      if (pos[i]-pos[i-1])>200 and (pos[i+1]-pos[i])>200:
        a = sig[pos[i]-150:pos[i]+150]
        a, cD3, cD2, cD1 = pywt.wavedec(a, 'db6', level=3)
        X.append(a)
        Y.append(labels.index(sym[i]))

In [0]:
X = np.array(X)
Y = np.array(Y)
print(X.shape)
print(Y.shape)
print(Counter(Y))

(78546, 47)
(78546,)
Counter({0: 60621, 1: 7772, 2: 7141, 3: 1585, 4: 1427})


In [0]:
data_len = len(X)
np.random.seed(42)
idx = list(range(data_len))
np.random.shuffle(idx)

In [0]:
train_len = int(data_len*0.8) #
valid_len = int(data_len*0.2)

X_train = X[idx][:train_len]
X_valid = X[idx][train_len:train_len+valid_len]
Y_train = Y[idx][:train_len]
Y_valid = Y[idx][train_len:train_len+valid_len]


print(X_train.shape)
print(X_valid.shape)
print(Counter(Y_train))
print(Counter(Y_valid))

fn = "data_"+"NLRAV"+".pk"
with open(fn, "wb") as fw:
    pk.dump(X_train, fw, protocol=pk.HIGHEST_PROTOCOL)
    pk.dump(Y_train, fw, protocol=pk.HIGHEST_PROTOCOL)
    pk.dump(X_valid, fw, protocol=pk.HIGHEST_PROTOCOL)
    pk.dump(Y_valid, fw, protocol=pk.HIGHEST_PROTOCOL)

(62836, 47)
(15709, 47)
Counter({0: 48390, 1: 6280, 2: 5744, 3: 1280, 4: 1142})
Counter({0: 12231, 1: 1491, 2: 1397, 3: 305, 4: 285})


In [0]:
import numpy as np
import pickle as pk
import os, sys
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score 
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam

def get_session(gpu_fraction=0.1):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)
    return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
  
K.set_session(get_session())



mode = "NLRAV" # NLRAV / NSVFQ
model_type = "BiLSTM"
save = "result_NLRAV_0_1" # result_NLRAV_0

fn = "data_"+mode+".pk"
with open(fn, "rb") as fp:
    X_train = pk.load(fp)
    Y_train = pk.load(fp)
    X_valid = pk.load(fp)
    Y_valid = pk.load(fp)
    

if model_type != 'Dense':
    X_train = np.expand_dims(X_train, axis=-1)
    X_valid = np.expand_dims(X_valid, axis=-1)

print(X_train.shape)
print(X_valid.shape)
f_size = X_train.shape[1]
class_num = 5

#============================================#

lr = 0.005
batch_size=128

Y_train = keras.utils.to_categorical(Y_train, num_classes=class_num)

def make_model():
    model = Sequential()
    if model_type == '1D':
        model.add(Conv1D(18, 7, activation='relu', input_shape=(f_size,1)))
        model.add(MaxPooling1D(2))
        model.add(Conv1D(18, 7, activation='relu'))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
    elif model_type == '1D-small':
        model.add(Conv1D(10, 3, activation='relu', input_shape=(f_size,1)))
        model.add(MaxPooling1D(2))
        model.add(Conv1D(10, 3, activation='relu'))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
    elif model_type == '1D-large':
        model.add(Conv1D(50, 13, activation='relu', input_shape=(f_size,1)))
        model.add(MaxPooling1D(2))
        model.add(Conv1D(50, 13, activation='relu'))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
    elif model_type == 'LSTM':
        model.add(LSTM(64, return_sequences=True, dropout=0.1, input_shape=(f_size, 1)))
        model.add(LSTM(32, return_sequences=True, dropout=0.1))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
    elif model_type == 'BiLSTM':
        model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.1), merge_mode='sum', input_shape=(f_size, 1)))
        model.add(Bidirectional(LSTM(32, return_sequences=True, dropout=0.1), merge_mode='sum'))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
    elif model_type == 'Dense':
        model.add(Dense(256, input_dim=f_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(256, input_dim=f_size, activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(class_num, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr))
    return model

model = make_model()
best_SE = 0
best_ACC = 0
best_model = make_model()
patience = 30
pcnt = 0

bin_label = lambda x: min(1,x)

for e in range(1, 300+1):

    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1, verbose=0)
    
    model.save_weights("weights_bilstm.h5")
    
    y_pred = model.predict(X_valid)
    y_pred = np.argmax(y_pred, axis=1)
    acc = np.sum(y_pred==Y_valid)/len(Y_valid)

    y_true = list(map(bin_label, Y_valid))
    y_pred = list(map(bin_label, y_pred))
    auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    SE = tp/(tp+fn)
    SP = tn/(fp+tn)

    if SE+acc > best_SE+best_ACC:
        best_SE, best_ACC = SE, acc
        best_model.set_weights(model.get_weights())
        pcnt = 0
    else:
        pcnt += 1
    
    print("Epoch: %d | SE: %.4f | Best SE: %.4f | ACC: %.4f | Best ACC: %.4f | AUC: %.4f | SP: %.4f" %(e, SE, best_SE, acc, best_ACC, auc, SP))

    


(62836, 47, 1)
(15709, 47, 1)
Epoch: 1 | SE: 0.9721 | Best SE: 0.9721 | ACC: 0.9880 | Best ACC: 0.9880 | AUC: 0.9835 | SP: 0.9949
Epoch: 2 | SE: 0.9793 | Best SE: 0.9793 | ACC: 0.9915 | Best ACC: 0.9915 | AUC: 0.9887 | SP: 0.9980
Epoch: 3 | SE: 0.9842 | Best SE: 0.9842 | ACC: 0.9941 | Best ACC: 0.9941 | AUC: 0.9911 | SP: 0.9980
Epoch: 4 | SE: 0.9853 | Best SE: 0.9853 | ACC: 0.9949 | Best ACC: 0.9949 | AUC: 0.9919 | SP: 0.9985
Epoch: 5 | SE: 0.9773 | Best SE: 0.9853 | ACC: 0.9934 | Best ACC: 0.9949 | AUC: 0.9882 | SP: 0.9992


KeyboardInterrupt: ignored

In [0]:
model.load_weights("weights_bilstm.h5")